In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_data = pd.read_csv('train-data.tsv', sep='\t', header=None, names=['label', 'message'])
valid_data = pd.read_csv('valid-data.tsv', sep='\t', header=None, names=['label', 'message'])
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
valid_data['label'] = valid_data['label'].map({'ham': 0, 'spam': 1})

In [ ]:
# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data['message'])

# Convert messages to sequences
train_sequences = tokenizer.texts_to_sequences(train_data['message'])
valid_sequences = tokenizer.texts_to_sequences(valid_data['message'])

# Pad sequences to make all of them same length
max_length = 100
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post')
valid_padded = tf.keras.preprocessing.sequence.pad_sequences(valid_sequences, maxlen=max_length, padding='post')

# Extract labels
train_labels = train_data['label'].values
valid_labels = valid_data['label'].values

In [ ]:
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(valid_padded, valid_labels), verbose=2)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  sequence = tokenizer.texts_to_sequences([pred_text])
  padded = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
  prediction = model.predict(padded)[0][0]
  label = 'spam' if prediction > 0.5 else 'ham'
  return [float(prediction), label]

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
# Test one sample
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
